# raw data에서 Investment 변수 생성에 필요한 info만 걸러내기

### raw data(investment)에서 필요한 정보

1. amount(이미 완료)
2. investor type
3. investment date (이미 완료) 
4. investor country
5. investor state (US 한정)
6. company(assignee) founded year
7. company(assignee) age
8. company(assignee) country
9. company state (US 한정)

In [1]:
import pandas as pd
import zipfile
import csv
import os

os.chdir("E:/patent")

In [2]:
zf_patent1 = zipfile.ZipFile('veh_investment.zip') 
csv_patent1 = zf_patent1.open('veh_investment.csv')
csv_patent_reader1 = csv.reader(csv_patent1)
header = csv_patent_reader1.next()

In [3]:
for i in range(0,97):
    print i, header[i]

0 -i0t0
1 Age at Financing (Months)-i0t0
2 Company City-i0t0
3 Company Current Operating Stage-i0t0
4 Company Current Public Status-i0t0
5 Company Founded Year-i0t0
6 Company ID-i0t0
7 Company IPO Date-i0t0
8 Company Investment Stage 1 at Round Date-i0t0
9 Company Investment Stage 2 at Round Date-i0t0
10 Company Investment Stage 3 at Round Date-i0t0
11 Company MoneyTree Region-i0t0
12 Company Name-i0t0
13 Company Nation-i0t0
14 Company Number of Employees-i0t0
15 Company State / Region-i0t0
16 Company Status-i0t0
17 Company VE Primary Industry Class-i0t0
18 Company VE Primary Industry Major Group-i0t0
19 Company World Location (Branch Office)-i0t0
20 Company Zip Code-i0t0
21 Cumulative Dividends-i0t0
22 Deal Value (USD Mil)-i0t0
23 Debt Amount (USD Mil)-i0t0
24 Equity Amount Disclosed (USD Mil)-i0t0
25 Equity Amount Estimated (USD Mil)-i0t0
26 Firm - First Investment Date-i0t0
27 Firm - Last Investment Date-i0t0
28 Firm - Total Number of Deals-i0t0
29 Firm Affiliations-i0t0
30 Firm Cap

In [5]:
firm_col = []
for i in range(0,len(header)):
    if 'Firm Name' in header[i]:   
        firm_col.append(i)

In [7]:
firm = []
assignee = []
amt = []
date = []
inv_type = []
fnd_type = []
inv_country = []
inv_state = []
ass_fyear = []
ass_age = []
ass_country = []
ass_state = []

for line in csv_patent_reader1:
    for i in firm_col:
        if line[i]=='':
            break

        else:
            firm.append(line[i])
            assignee.append(line[i-24])
            amt.append(line[i+15])
            date.append(line[i+33])
            inv_type.append(line[i+8])
            fnd_type.append(line[i+30])
            inv_country.append(line[i+1])
            inv_state.append(line[i+4])
            ass_fyear.append(line[i-31])
            ass_age.append(line[i-35])
            ass_country.append(line[i-23])
            ass_state.append(line[i-21])

In [17]:
df = pd.DataFrame({'firm':firm,'assignee':assignee,'amount':amt,'date':date,'firm_type':inv_type,'firm_country':inv_country,
                   'firm_state':inv_state,'assignee_founded_year':ass_fyear,'assignee_age_month':ass_age,'assignee_country':ass_country,
                  'assignee_state':ass_state,'fund_type':fnd_type})

In [18]:
df.to_csv('veh_selected_deal_info_duplicates.csv',index=False)

In [19]:
cols=list(df)
del cols[-1]
cols

['amount',
 'assignee',
 'assignee_age_month',
 'assignee_country',
 'assignee_founded_year',
 'assignee_state',
 'date',
 'firm',
 'firm_country',
 'firm_state',
 'firm_type']

In [22]:
df_nodups = df.drop_duplicates(subset=cols)

In [23]:
len(df_nodups.index)

89007

In [24]:
df_nodups.to_csv('veh_sel_info_temp.csv',index=False)

missing value, 변수 type 처리 필요   
('-' -> NaN 처리, str -> float 처리)

In [17]:
def bye_missing_numeric(df,col):
    for i in range(0,len(df.index)):
        for j in col:
            if df.iloc[i,j] == '-' or df.iloc[i,j] == '':
                df.iloc[i,j] = np.nan
                
def bye_missing_string(df,col):
    for i in range(0,len(df.index)):
        for j in col:
            if df.iloc[i,j] == '-' or df.iloc[i,j] == '':
                df.iloc[i,j] = ''

In [30]:
import numpy as np

col = range(0,11)
num_col = [0,2,4]
str_col = [x for x in col if x not in num_col]
bye_missing_numeric(df,num_col)
bye_missing_string(df,str_col)

In [33]:
df.iloc[0:5,]

,amount,assignee,assignee_age_month,assignee_country,assignee_founded_year,assignee_state,date,firm,firm_country,firm_state,firm_type
0,90.3,Singer Company,NaN,United States,NaN,Florida,1987-10-29,Bilzerian Partners Lp,United States,Florida,Private Equity Firm
1,NaN,Singer Company,NaN,United States,NaN,Florida,1988-04-25,Bilzerian Partners Lp,United States,Florida,Private Equity Firm
2,0.045,"Wulfsberg Electronics, Inc.",117,United States,1970,Kansas,1979-10-01,GT Technology Fund,United Kingdom,Non-US,Bank Affiliated
3,0.09,"Wulfsberg Electronics, Inc.",117,United States,1970,Kansas,1979-10-01,Melchor Venture Management Inc,United States,California,Private Equity Firm
4,1.643,"Wulfsberg Electronics, Inc.",117,United States,1970,Kansas,1979-10-01,Hewlett-Packard Profit Sharing Trust,United States,California,"Endowment, Foundation or Pension Fund"


In [36]:
df.iloc[:,0]=map(float,df.iloc[:,0])

In [50]:
for i in range(0,len(df.index)):
    for j in [2,4]:
        if pd.isnull(df.iloc[i,j])==False:
            df.iloc[i,j]=int(df.iloc[i,j])

In [51]:
print type(df.iloc[0,0]), type(df.iloc[0,2]), type(df.iloc[2,2]), type(df.iloc[2,4])

<type 'numpy.float64'> <type 'float'> <type 'int'> <type 'int'>


In [52]:
def nameprocessing(s):
    s = str(s)
    
    s = s.replace(".","")
    s = s.replace(",","")    
    s = s.replace(";","")
    s = ''.join(s.split()).lower()
    
    s = s.replace("incorporated","inc")
    s = s.replace("limited","ltd")
    s = s.replace("gmbh","")
    
    rm=['inc','ltd','llc','pte']
    for i in range(0,len(s)):
        if s[-3:] in rm:
            s=s[:-3]
        else:
            break

    return s

In [53]:
df['new_assignee']=''
df.iloc[0:5,]

,amount,assignee,assignee_age_month,assignee_country,assignee_founded_year,assignee_state,date,firm,firm_country,firm_state,firm_type,new_assignee
0,90.300,Singer Company,NaN,United States,NaN,Florida,1987-10-29,Bilzerian Partners Lp,United States,Florida,Private Equity Firm,
1,NaN,Singer Company,NaN,United States,NaN,Florida,1988-04-25,Bilzerian Partners Lp,United States,Florida,Private Equity Firm,
2,0.045,"Wulfsberg Electronics, Inc.",117,United States,1970,Kansas,1979-10-01,GT Technology Fund,United Kingdom,Non-US,Bank Affiliated,
3,0.090,"Wulfsberg Electronics, Inc.",117,United States,1970,Kansas,1979-10-01,Melchor Venture Management Inc,United States,California,Private Equity Firm,
4,1.643,"Wulfsberg Electronics, Inc.",117,United States,1970,Kansas,1979-10-01,Hewlett-Packard Profit Sharing Trust,United States,California,"Endowment, Foundation or Pension Fund",


In [56]:
for i in range(0,len(df.index)):
    df.iloc[i,11]=nameprocessing(df.iloc[i,1])

In [58]:
df.to_csv('veh_selected_deal_info.csv',index=False)